### Utils

In [34]:
import base64
import os
import json
from datetime import datetime
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage
import requests
import re
import json
from typing import Dict, Any

def load_markdown_to_str(file_path):
	with open(file_path, 'r', encoding='utf-8') as md_file:
		markdown_content = md_file.read()
	return markdown_content

def load_latest_sprint_status(base_path):
    """
    Find the latest sprint directory and load the project-sprint-status.md file.
    
    Args:
    base_path (str): Path to the directory containing sprint folders.
    
    Returns:
    str: Content of the project-sprint-status.md file from the latest sprint.
    """
    try:
        # List all directories in the base path
        directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
        
        # Filter and sort sprint directories
        sprint_dirs = sorted([d for d in directories if d.startswith('sprint') and d[6:].isdigit()],
                             key=lambda x: int(x[6:]),
                             reverse=True)
        
        if not sprint_dirs:
            return "No sprint directories found."
        
        # Get the latest sprint directory
        latest_sprint = sprint_dirs[0]
        sprint_path = os.path.join(base_path, latest_sprint)
        
        # Look for project-sprint-status.md in the latest sprint directory
        status_file = os.path.join(sprint_path, 'project-sprint-status.md')
        
        if os.path.exists(status_file):
            with open(status_file, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            return f"project-sprint-status.md not found in {latest_sprint}."
    
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
    
def load_latest_sprint_backlog(base_path):
    """
    Find the latest sprint directory and load the project-sprint-backlog.json file.
    
    Args:
    base_path (str): Path to the directory containing sprint folders.
    
    Returns:
    dict: Content of the project-sprint-backlog.json file from the latest sprint.
    """
    try:
        # List all directories in the base path
        directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
        
        # Filter and sort sprint directories
        sprint_dirs = sorted([d for d in directories if d.startswith('sprint') and d[6:].isdigit()],
                             key=lambda x: int(x[6:]),
                             reverse=True)
        
        if not sprint_dirs:
            return {"error": "No sprint directories found."}
        
        # Get the latest sprint directory
        latest_sprint = sprint_dirs[0]
        sprint_path = os.path.join(base_path, latest_sprint)
        
        # Look for project-sprint-backlog.json in the latest sprint directory
        backlog_file = os.path.join(sprint_path, 'project-sprint-backlog.json')
        
        if os.path.exists(backlog_file):
            with open(backlog_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        else:
            return {"error": f"project-sprint-backlog.json not found in {latest_sprint}."}
    
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}
    
def export_transcript(state, folder_path):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    transcript = state["transcript"]
    
    filename = state["meeting_type"].replace(" ","_") + str(id) + ".txt"
    
    os.makedirs(os.path.join(folder_path), exist_ok=True)
    
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the string to a text file
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(transcript)

def export_state(state, folder_path, filename):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    
    filename = filename+str(id)+".json"
    
    os.makedirs(folder_path, exist_ok=True)
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the state dict to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(state, f, indent=4)

def load_txt_to_str(file_path):
    with open(file_path, 'r', encoding='utf-8') as txt_file:
        text_content = txt_file.read()
    return text_content

def load_from_json(file_path):
    """
    Load data from a JSON file.
    
    Args:
    file_path (str): Path to the JSON file.
    
    Returns:
    dict: A dictionary containing the loaded JSON data.
    """
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file {file_path}")
        return {}
    
def render_mermaid_diagram(diagram_code: str) -> str:
    # Encode the Mermaid code
    encoded_diagram = base64.b64encode(diagram_code.encode('utf-8')).decode('utf-8')
    
    # Make a request to the Mermaid rendering service
    url = f"https://mermaid.ink/img/{encoded_diagram}"
    response = requests.get(url)
    
    if response.status_code == 200:
        # Return the URL of the rendered image
        return url
    else:
        # If rendering failed, return the original Mermaid code
        return f"```mermaid\n{diagram_code}\n```"
    
def format_mermaid(input_string):
    # Step 1: Remove redundant "```mermaid" at the start and end
    cleaned_string = input_string.replace('```mermaid', '').replace('```', '')
    
    # Step 2: Replace escaped newlines with actual newlines
    formatted_string = cleaned_string.replace(r'\n', '\n')
    
    # Step 3: Strip any leading/trailing whitespace
    formatted_string = formatted_string.strip()

    return formatted_string	


def export_meeting_history(state, output_file='meeting_history.json'):
    """
    Export the meeting history to a JSON file.
    
    Args:
    state (dict): The state dictionary containing the meeting history.
    output_file (str): The name of the output file. Defaults to 'meeting_history.json'.
    
    Returns:
    None
    """
    meeting_history = state.get("meeting_history", [])
    
    # Ensure meeting_history is a list
    if not isinstance(meeting_history, list):
        meeting_history = [meeting_history]
    
    try:
        with open(output_file, 'w') as file:
            json.dump(meeting_history, file, indent=2)
        print(f"Meeting history exported successfully to {output_file}")
    except IOError:
        print(f"Error: Unable to write to file {output_file}")

    return state  # Return the state to maintain consistency with your workflow


def manage_sprint_folders(state, project_folder):
    """
    Manages sprint folders based on the meeting type.
    Creates a new sprint folder if necessary and adds required files.
    
    :param state: The current state dictionary
    :param project_folder: Path to the project folder
    :return: Updated state with new sprint information
    """
    if "planning" in state.get("meeting_type", "").lower() or "plan" in state.get("meeting_type", "").lower():
        # List all directories in the project folder
        directories = [d for d in os.listdir(project_folder) if os.path.isdir(os.path.join(project_folder, d))]
        
        # Filter and find the highest sprint number
        sprint_numbers = [int(re.findall(r'\d+', d)[0]) for d in directories if d.startswith("sprint") and re.findall(r'\d+', d)]
        
        if sprint_numbers:
            new_sprint_number = max(sprint_numbers) + 1
        else:
            new_sprint_number = 1
        
        # Create new sprint folder
        new_sprint_folder = os.path.join(project_folder, f"sprint{new_sprint_number}")
        os.makedirs(new_sprint_folder, exist_ok=True)
        
        # Create project_sprint_status.md
        status_file_path = os.path.join(new_sprint_folder, "project_sprint_status.md")
        with open(status_file_path, 'w') as status_file:
            status_file.write(f"# Sprint {new_sprint_number} Status\n\nStatus details will be updated here.")
        
        # Create project_sprint_backlog.json
        backlog_file_path = os.path.join(new_sprint_folder, "project_sprint_backlog.json")
        initial_backlog = {}
        with open(backlog_file_path, 'w') as backlog_file:
            json.dump(initial_backlog, backlog_file, indent=2)
        
        # Update state with new sprint information
        state["current_sprint_number"] = new_sprint_number
        state["current_sprint_folder"] = new_sprint_folder
        state["sprint_status_file"] = status_file_path
        state["sprint_backlog_file"] = backlog_file_path
        
        print(f"Created new sprint folder: {new_sprint_folder}")
    else:
        print("Meeting type does not indicate a planning session. No new sprint folder created.")
    
    return state


def load_json(file_path: str) -> Dict[str, Any]:
    """
    Reads a JSON file and returns its contents as a dictionary.

    :param file_path: The path to the JSON file to be read
    :return: A dictionary containing the data from the JSON file
    :raises FileNotFoundError: If the specified file does not exist
    :raises json.JSONDecodeError: If the file is not valid JSON
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        print(f"Successfully loaded JSON from {file_path}")
        return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading {file_path}: {str(e)}")
        raise


def get_latest_sprint_folder(project_folder: str) -> str:
    """
    Scans the project folder for sprint folders and returns the name of the latest sprint folder.

    :param project_folder: Path to the project folder (e.g., 'project1/')
    :return: Name of the latest sprint folder (e.g., 'sprint5'), or None if no sprint folders are found
    """
    # List all items in the project folder
    items = os.listdir(project_folder)

    # Filter for sprint folders and extract their numbers
    sprint_folders = []
    for item in items:
        if os.path.isdir(os.path.join(project_folder, item)):
            match = re.match(r'sprint(\d+)', item, re.IGNORECASE)
            if match:
                sprint_number = int(match.group(1))
                sprint_folders.append((item, sprint_number))

    # Sort sprint folders by number (descending) and return the latest
    if sprint_folders:
        latest_sprint = max(sprint_folders, key=lambda x: x[1])
        print(f"Latest sprint folder found: {latest_sprint[0]}")
        return latest_sprint[0]
    else:
        print("No sprint folders found.")
        return None


def export_markdown(content, file_path):
    """
    Export content to a Markdown file.
    
    Args:
    content (str): The content to be written to the file.
    file_path (str): The path where the file should be saved.
    """
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"Successfully exported Markdown to {file_path}")
    except Exception as e:
        print(f"Error exporting Markdown: {str(e)}")

def export_json(content, file_path):
    """
    Export content to a JSON file.
    
    Args:
    content (dict): The content to be written to the file.
    file_path (str): The path where the file should be saved.
    """
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(content, f, indent=2)
        print(f"Successfully exported JSON to {file_path}")
    except Exception as e:
        print(f"Error exporting JSON: {str(e)}")
        
def load_latest_json(folder_path: str) -> Dict[str, Any]:
    """
    Reads the latest JSON file from a given folder and returns its contents as a dictionary.

    :param folder_path: The path to the folder containing JSON files
    :return: A dictionary containing the data from the latest JSON file
    :raises FileNotFoundError: If no JSON files are found in the specified folder
    :raises json.JSONDecodeError: If the latest file is not valid JSON
    """
    try:
        # Get all JSON files in the folder
        json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
        
        if not json_files:
            raise FileNotFoundError(f"No JSON files found in {folder_path}")
        
        # Find the latest JSON file
        latest_file = max(json_files, key=lambda f: os.path.getmtime(os.path.join(folder_path, f)))
        latest_file_path = os.path.join(folder_path, latest_file)
        
        # Read and parse the latest JSON file
        with open(latest_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        print(f"Successfully loaded latest JSON from {latest_file_path}")
        return data
    
    except FileNotFoundError:
        print(f"Error: No JSON files found in {folder_path}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {latest_file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading from {folder_path}: {str(e)}")
        raise
    
def load_second_latest_json(folder_path: str) -> Dict[str, Any]:
    """
    Reads the second latest JSON file from a given folder and returns its contents as a dictionary.

    :param folder_path: The path to the folder containing JSON files
    :return: A dictionary containing the data from the second latest JSON file
    :raises FileNotFoundError: If fewer than two JSON files are found in the specified folder
    :raises json.JSONDecodeError: If the second latest file is not valid JSON
    """
    try:
        # Get all JSON files in the folder
        json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
        
        if len(json_files) < 2:
            raise FileNotFoundError(f"Fewer than two JSON files found in {folder_path}")
        
        # Sort JSON files by modification time, newest first
        sorted_files = sorted(json_files, key=lambda f: os.path.getmtime(os.path.join(folder_path, f)), reverse=True)
        
        # Get the second latest file
        second_latest_file = sorted_files[1]
        second_latest_file_path = os.path.join(folder_path, second_latest_file)
        
        # Read and parse the second latest JSON file
        with open(second_latest_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        print(f"Successfully loaded second latest JSON from {second_latest_file_path}")
        return data
    
    except FileNotFoundError:
        print(f"Error: Fewer than two JSON files found in {folder_path}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {second_latest_file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading from {folder_path}: {str(e)}")
        raise 
def load_employee_profiles(file_path):
    """
    Load employee profiles from a JSON file.
    
    Args:
    file_path (str): Path to the JSON file containing employee profiles.
    
    Returns:
    dict: A dictionary of employee profiles.
    """
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file {file_path}")
        return {}

### Prompts

In [35]:
GENERAL_INFO_EXTRACTOR_PROMPT = """
You are an AI assistant helping to extract general information from a daily scrum meeting note. Your task is to summarize the key points that are relevant to all team members.


Here's the daily scrum meeting note:
####
{note}
####

Please extract and summarize the following information:
1. Overall sprint progress
2. Main challenges or blockers
3. Key decisions made
4. Any updates that affect the entire team

Provide this information in a concise, bullet-point format.
IMPORTANT: Answer only with the general information. No more. 
You must not say here is the summary ... just the information.
Note that your task is not to make a summary but to find information that effect everyone.
"""

PERSONAL_TASK_EXTRACTOR_PROMPT = """
You are an AI assistant helping to extract personal tasks and progress for a specific team member from a daily scrum meeting note.

Here's the daily scrum meeting note:
####
{note}
####

Here's the profile of the team member:
####
{employee_profile}
####

Here's the curret state of the project: 
Note that this document already contains information about the current meeting that have just happened. It is up to date.
####
{sprint_state}
####

Here's the current state of the backlog:
Note that this document does not contains information about the current meeting. It representes a previous state.
####
{sprint_backlog}
####

Please extract the following information for this team member:
1. Their reported progress since the last meeting
2. Their current tasks or focus areas
3. Any blockers or challenges they're facing
4. Their next steps or goals

Provide this information in a concise, bullet-point format.
"""


DIAGRAM_GENERATOR_PROMPT = """
You are an AI assistant tasked with generating a Mermaid diagram based on a team member's tasks and progress.

Here's the daily scrum meeting note:
####
{note}
####


Here's the information about the team member's tasks and progress:
####
{personal_tasks}
####

Please create a Mermaid diagram that visualizes this information. The diagram should show:
1. Completed tasks
2. In-progress tasks
3. Upcoming tasks
4. Any dependencies between tasks

Provide only the Mermaid diagram code, without any explanation or additional text.
"""


REPORT_COMPILER_PROMPT = """
You are an AI assistant tasked with compiling a personalized daily scrum report for a team member.

Here's the general project information:
####
{general_info}
####

Here's the team member's personal task information:
####
{personal_tasks}
####

Here's a rendered Mermaid diagram representing the team member's tasks:
![Task Diagram]({diagram})

Please compile a personalized report for this team member. The report should include:
1. A brief overview of the sprint progress
2. The team member's personal progress and next steps
3. Any relevant challenges or blockers
4. The rendered Mermaid diagram (include it using the Markdown image syntax)

Format the report in Markdown, including appropriate headers and sections.
"""


REPORT_COMPILER_PROMPT = """
You are an AI assistant tasked with compiling a personalized daily scrum report for a team member.

Here's the general project information:
####
{general_info}
####

Here's the team member's personal task information:
####
{personal_tasks}
####

Here's a rendered Mermaid diagram representing the team member's tasks:
![Task Diagram]({diagram})

Please compile a personalized report for this team member. The report should include:
1. A brief overview of the sprint progress
2. The team member's personal progress and next steps
3. Any relevant challenges or blockers
4. The rendered Mermaid diagram (include it using the Markdown image syntax)

Format the report in Markdown, including appropriate headers and sections.
"""

MOTIVATIONAL_MESSAGE_PROMPT = """
You are an AI assistant tasked with generating a short, personalized motivational message or fun fact for a team member.

Here's the team member's profile:
####
{employee_profile}
####

Please generate a short, uplifting message or interesting fun fact related to the team member's interests, role, or recent achievements. The message should be positive and encouraging.

Provide only the message, without any additional explanation.
"""



### State

In [36]:

from typing import TypedDict

class DailyState(TypedDict):
    sprint_state: str
    sprint_backlog: str
    employee_profiles: Dict[str, str]
    note: str
    general_info : str
    reports: Dict[str, str]  # Key: employee name, Value: report content
    diagrams: Dict[str, str]  # Key: employee name, Value: Mermaid diagram code
    personal_tasks : Dict [str,str]
    
from abc import ABC, abstractmethod

class Graph(ABC):
    @abstractmethod
    def create_graph(self):
        pass
    
    @abstractmethod
    def run_graph(self, project_folder):
        pass

### Node functions

In [37]:
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
from  dotenv import load_dotenv
import os

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
#model = ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key)
model = ChatAnthropic(model="claude-3-5-sonnet-20240620", anthropic_api_key=anthropic_api_key, max_tokens= 8192)
def general_info_extractor_node(state: DailyState) -> DailyState:
    formatted_prompt = GENERAL_INFO_EXTRACTOR_PROMPT.format(
        note=state["note"]
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Extract the general information from the daily scrum note.")
    ]
    response = model.invoke(messages)
    state["general_info"] = response.content
    return state

def personal_task_extractor_node(state: DailyState) -> DailyState:
    for employee, profile in state["employee_profiles"].items():
        formatted_prompt = PERSONAL_TASK_EXTRACTOR_PROMPT.format(
            note=state["note"],
            employee_profile=profile,
            sprint_state=state["sprint_state"],
            sprint_backlog=state["sprint_backlog"]
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content=f"Extract personal tasks for {employee}.")
        ]
        response = model.invoke(messages)
        state["personal_tasks"][employee] = response.content
    return state

def diagram_generator_node(state: DailyState) -> DailyState:
    for employee, tasks in state["personal_tasks"].items():
        formatted_prompt = DIAGRAM_GENERATOR_PROMPT.format(
            note=state["note"],
            personal_tasks=tasks
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content=f"Generate a Mermaid diagram for {employee}'s tasks.")
        ]
        response = model.invoke(messages)
        state["diagrams"][employee] = response.content
    return state

def report_compiler_node(state: DailyState) -> DailyState:
    for employee in state["employee_profiles"].keys():
        rendered_diagram = render_mermaid_diagram(format_mermaid(state["diagrams"][employee]))
        formatted_prompt = REPORT_COMPILER_PROMPT.format(
            general_info=state["general_info"],
            personal_tasks=state["personal_tasks"][employee],
            diagram=rendered_diagram
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content=f"Compile a personalized report for {employee}.")
        ]
        response = model.invoke(messages)
        state["reports"][employee] = response.content
    return state

def motivational_message_node(state: DailyState) -> DailyState:
    for employee, profile in state["employee_profiles"].items():
        formatted_prompt = MOTIVATIONAL_MESSAGE_PROMPT.format(
            employee_profile=profile
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content=f"Generate a motivational message for {employee}.")
        ]
        response = model.invoke(messages)
        state["reports"][employee] += f"\n\n{response.content}"
    return state

### Graph

In [28]:
from langgraph.graph import StateGraph, END
class UpdateProjectState(Graph):
    def __init__(self):
        self.workflow = None
        
    def create_graph(self):
        workflow = StateGraph(DailyState)
    
        workflow.add_node("general_info_extractor", general_info_extractor_node)
        workflow.add_node("personal_task_extractor", personal_task_extractor_node)
        workflow.add_node("diagram_generator", diagram_generator_node)
        workflow.add_node("report_compiler", report_compiler_node)
        workflow.add_node("motivational_message", motivational_message_node)
    
        workflow.add_edge("general_info_extractor", "personal_task_extractor")
        workflow.add_edge("personal_task_extractor", "diagram_generator")
        workflow.add_edge("diagram_generator", "report_compiler")
        workflow.add_edge("report_compiler", "motivational_message")
        workflow.add_edge("motivational_message", END)
        
        workflow.set_entry_point("general_info_extractor")
    
        self.workflow = workflow.compile()
        return self.workflow
    
    def run_graph(self, project_folder: str, meeting_note: str, meeting_type: str) -> DailyState:
        if self.workflow is None:
            raise ValueError("Graph has not been created. Call create_graph() first.")
        
        state = initialize_project_update_state(
            project_folder=project_folder
        )
        
        #updated_state = self.workflow.invoke(state)
        
        #self.export_updated_state(updated_state, project_folder)
        return state

    def export_state(self, state: DailyState, project_folder: str):
        export_state(state, folder_path= os.path.join(project_folder, "state-logs"), filename="daily_report")
        for employee, report in state["reports"].items():
            export_markdown(report, os.path.join(project_folder, "reports", f"{employee}_report_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.md"))

def initialize_project_update_state(project_folder: str) -> DailyState:
    update_state = load_latest_json(folder_path=os.path.join(project_folder, "state-logs"))
    note_taker_state=load_second_latest_json(folder_path=os.path.join(project_folder,"state-logs"))
    return DailyState(
        sprint_state=update_state["updated_project_state"],
        sprint_backlog=update_state["updated_project_backlog"],
        employee_profiles=load_employee_profiles(os.path.join(project_folder, "employee-profiles.json")),
        note=note_taker_state["note_draft"],
        general_info="",
        reports={},
        diagrams={},
        personal_tasks={}
    )


### Testing

In [21]:
project_folder="../../../data_/project1/"
state = initialize_project_update_state(project_folder)
state

Successfully loaded latest JSON from ../../../data_/project1/state-logs\update_state3.json
Successfully loaded second latest JSON from ../../../data_/project1/state-logs\note_taker2.json


{'sprint_state': 'Here\'s the updated project state markdown based on the meeting analysis:\n\n```markdown\n# HealthTrack Pro - Project State\n\n## Overall Project Progress\n10% complete (initial project setup and planning phase)\n\n## Current Phase\nPhase 1 (MVP Development) - Just initiated\n\n## Key Milestones and Status\n1. Project Kickoff - Completed (June 17, 2024)\n2. Initial Sprint Planning - Completed (June 17, 2024)\n3. Development Environment Setup - In Progress\n4. User Authentication and Profile Management - Planned for first sprint\n5. Basic Health Metrics Dashboard - Planned for upcoming sprints\n6. Activity Tracking - Planned for upcoming sprints\n7. Simplified Nutrition Logging - Planned for upcoming sprints\n\n## High-level Risks or Blockers\n1. Potential security and compliance requirements (e.g., HIPAA) for health data - Under investigation\n\n## Team Morale/Health Indicator\nGood - Team roles are clearly defined, and there\'s a strong focus on collaboration and pro

In [39]:
graph = UpdateProjectState()
graph.create_graph()
print(graph.workflow.get_graph().draw_mermaid())


%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	general_info_extractor(general_info_extractor)
	personal_task_extractor(personal_task_extractor)
	diagram_generator(diagram_generator)
	report_compiler(report_compiler)
	motivational_message(motivational_message)
	__end__([<p>__end__</p>]):::last
	__start__ --> general_info_extractor;
	diagram_generator --> report_compiler;
	general_info_extractor --> personal_task_extractor;
	motivational_message --> __end__;
	personal_task_extractor --> diagram_generator;
	report_compiler --> motivational_message;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [23]:
general_info_extractor_node(state)


{'sprint_state': 'Here\'s the updated project state markdown based on the meeting analysis:\n\n```markdown\n# HealthTrack Pro - Project State\n\n## Overall Project Progress\n10% complete (initial project setup and planning phase)\n\n## Current Phase\nPhase 1 (MVP Development) - Just initiated\n\n## Key Milestones and Status\n1. Project Kickoff - Completed (June 17, 2024)\n2. Initial Sprint Planning - Completed (June 17, 2024)\n3. Development Environment Setup - In Progress\n4. User Authentication and Profile Management - Planned for first sprint\n5. Basic Health Metrics Dashboard - Planned for upcoming sprints\n6. Activity Tracking - Planned for upcoming sprints\n7. Simplified Nutrition Logging - Planned for upcoming sprints\n\n## High-level Risks or Blockers\n1. Potential security and compliance requirements (e.g., HIPAA) for health data - Under investigation\n\n## Team Morale/Health Indicator\nGood - Team roles are clearly defined, and there\'s a strong focus on collaboration and pro

In [24]:
personal_task_extractor_node(state)
state["personal_tasks"]


{'Sarah Chen': "Based on the meeting notes and Sarah Chen's profile as the Project Manager/Scrum Master, here's the extracted information:\n\n1. Reported progress since last meeting:\n   • Led the project kickoff and initial sprint planning meeting\n   • Facilitated team discussions on MVP features prioritization\n   • Established sprint duration and initial velocity\n\n2. Current tasks or focus areas:\n   • Setting up Confluence space for project documentation\n   • Sending calendar invites for recurring meetings (daily stand-ups, sprint reviews, retrospectives)\n   • Overseeing the development environment setup process\n\n3. Blockers or challenges:\n   • No specific blockers mentioned for Sarah in the meeting notes\n\n4. Next steps or goals:\n   • Complete setup of Confluence space for project documentation\n   • Finalize and distribute all necessary meeting invites\n   • Schedule a technical planning session for database schema and architecture discussion\n   • Monitor progress on i

In [25]:
diagram_generator_node(state=state)
state["diagrams"]


{'Sarah Chen': '```mermaid\ngraph TD\n    subgraph Completed\n        A[Led project kickoff meeting]\n        B[Facilitated MVP features prioritization]\n        C[Established sprint duration and velocity]\n    end\n\n    subgraph In Progress\n        D[Setting up Confluence space]\n        E[Sending calendar invites]\n        F[Overseeing dev environment setup]\n    end\n\n    subgraph Upcoming\n        G[Schedule technical planning session]\n        H[Monitor initial project setup]\n        I[Prepare for daily stand-ups]\n    end\n\n    A --> D\n    B --> D\n    C --> D\n    D --> G\n    E --> I\n    F --> H\n    G --> H\n```',
 'Alex Rodriguez': '```mermaid\ngraph TD\n    subgraph Completed\n    end\n\n    subgraph In_Progress\n        A[Lead initial project structure setup]\n        B[Create repositories]\n        C[Configure ESLint and Prettier]\n    end\n\n    subgraph Upcoming\n        D[Prepare for user authentication implementation]\n        E[Participate in technical planning

In [26]:
report_compiler_node(state)
state["reports"]

{'Sarah Chen': "# Daily Scrum Report for Sarah Chen\n\n## Sprint Overview\nThe HealthTrack Pro project has successfully kicked off with a clear focus on MVP features for the initial 2-week sprint. The team has aligned on prioritizing user authentication, activity tracking, a basic health metrics dashboard, and simplified nutrition logging. The technology stack has been defined, and initial project structure is underway.\n\n## Personal Progress and Next Steps\n\n### Completed Tasks\n- Led the project kickoff and initial sprint planning meeting\n- Facilitated team discussions on MVP features prioritization\n- Established sprint duration and initial velocity\n\n### In Progress\n- Setting up Confluence space for project documentation\n- Sending calendar invites for recurring meetings (daily stand-ups, sprint reviews, retrospectives)\n- Overseeing the development environment setup process\n\n### Upcoming Tasks\n- Complete setup of Confluence space for project documentation\n- Finalize and d

In [27]:
motivational_message_node(state)
state["reports"]

{'Sarah Chen': "# Daily Scrum Report for Sarah Chen\n\n## Sprint Overview\nThe HealthTrack Pro project has successfully kicked off with a clear focus on MVP features for the initial 2-week sprint. The team has aligned on prioritizing user authentication, activity tracking, a basic health metrics dashboard, and simplified nutrition logging. The technology stack has been defined, and initial project structure is underway.\n\n## Personal Progress and Next Steps\n\n### Completed Tasks\n- Led the project kickoff and initial sprint planning meeting\n- Facilitated team discussions on MVP features prioritization\n- Established sprint duration and initial velocity\n\n### In Progress\n- Setting up Confluence space for project documentation\n- Sending calendar invites for recurring meetings (daily stand-ups, sprint reviews, retrospectives)\n- Overseeing the development environment setup process\n\n### Upcoming Tasks\n- Complete setup of Confluence space for project documentation\n- Finalize and d

In [31]:
graph.export_state(state, project_folder)

Successfully exported Markdown to ../../../data_/project1/reports\Sarah Chen_report_2024-10-05_20-46-28.md
Successfully exported Markdown to ../../../data_/project1/reports\Alex Rodriguez_report_2024-10-05_20-46-28.md
Successfully exported Markdown to ../../../data_/project1/reports\Emily Watson_report_2024-10-05_20-46-28.md
Successfully exported Markdown to ../../../data_/project1/reports\Michael Kim_report_2024-10-05_20-46-28.md
Successfully exported Markdown to ../../../data_/project1/reports\Olivia Martinez_report_2024-10-05_20-46-28.md
Successfully exported Markdown to ../../../data_/project1/reports\Liam Foster_report_2024-10-05_20-46-28.md


In [33]:
state["note"]

'Meeting Note: Project Kickoff and Initial Sprint Planning for HealthTrack Pro\n\nDate: June 17, 2024\nAttendees: Sarah Chen (Project Manager/Scrum Master), Alex Rodriguez (Senior Full-Stack Developer), Emily Watson (Frontend Developer), Michael Kim (Backend Developer), Olivia Martinez (QA Engineer/DevOps Specialist), Liam Foster (UI/UX Designer)\n\n1. Project Overview\n   - HealthTrack Pro: Comprehensive web application for personal health management\n   - Started on June 10, 2024\n   - Key components:\n     a. User Authentication and Profile Management\n     b. Activity Tracking\n     c. Nutrition Logging and Analysis\n     d. Health Metrics Dashboard\n     e. Goal Setting and Progress Tracking\n     f. Recommendation Engine\n     g. Social Features\n     h. Integration with popular fitness devices and apps\n\n2. Technology Stack\n   - Frontend: React.js, TypeScript, Tailwind CSS\n   - Backend: Node.js, Express.js, PostgreSQL\n   - DevOps: Docker, AWS, Jenkins\n   - Design: Figma, Ad

In [ ]:
s